# promptx

A framework for building AI systems.

```bash
pip install pxx
```

In [1]:
from promptx import prompt

character = 'Batman'
prompt(f'Write a character profile for {character}')

"NAME: Batman\n\nAGE: Typically portrayed as being in his late 30s to early 40s.\n\nOCCUPATION: Vigilante, Crime-fighter, Billionaire Philanthropist\n\nPERSONALITY: Batman is known for being mysterious, brooding, and relentless in his pursuit of justice. He is driven by a deep sense of vengeance and a desire to protect the innocent. Despite his tough exterior, Batman is also incredibly intelligent, strategic, and resourceful. He is a master of hand-to-hand combat and is always prepared with gadgets and technology to assist him in his mission.\n\nBACKSTORY: Batman's true identity is Bruce Wayne, a wealthy businessman who witnessed the murder of his parents as a child. This traumatic event fueled his obsession with fighting crime and inspired him to become Batman. Wayne honed his physical and mental abilities through years of training and travels around the world, learning from the best in various disciplines. He also utilizes his vast wealth to fund his crime-fighting operations and to 

By default, this returns a plain string response, but to generate complex data you can pass in the expected schema along with the prompt input.

*Note: `Entity` is a thin layer on top of `pydantic.BaseModel` that allows the object to be stored as an embedding. You can use `pydantic.BaseModel` directly if you don't need to store the object as an embedding and just want to use it as the prompt output schema.*

In [2]:
from pydantic import Field
from promptx.collection import Entity

class Character(Entity):
    name: str = Field(..., embed=False),
    description: str = Field(..., description='Describe the character in a few sentences')
    age: int = Field(..., ge=0, le=120)

batman = prompt('Generate a character profile for Batman', output=Character)
batman

/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)



Character(
    id='79c226a0-da19-4f81-9da8-cf20372e8199',
    type='character',
    name='Bruce Wayne',
    description='Bruce Wayne is the alter ego of Batman. He is a billionaire industrialist and philanthropist by day, and a crime-fighting vigilante by night. Bruce witnessed the murder of his parents as a child, which motivated him to devote his life to fighting crime and protecting Gotham City.',
    age=35
)

This returns an instance of the specified schema using the generated response as the input data. Let's create a list of instead.

In [3]:
characters = prompt(
    'Generate some characters from the Batman universe',
    output=[Character],
)

characters

/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


,id,type,name,description,age
0,b9feedb2-5847-4aec-8c0a-11d050bfb67a,character,Bruce Wayne,Bruce Wayne is the billionaire owner and CEO o...,38
1,5c421997-8e2b-41ac-af8a-bad8181a3555,character,Dick Grayson,Dick Grayson is Bruce Wayne's first ward and l...,25
2,9ab8ac13-408b-4b9a-b418-fe48ae1af899,character,Barbara Gordon,Barbara Gordon is the daughter of Gotham City ...,29
3,34233dc3-b115-4f76-b648-35fa41e2cc3f,character,Selina Kyle,"Selina Kyle, also known as Catwoman, is a skil...",35
4,180cfd37-efcb-4431-9334-c87e246030f6,character,Harvey Dent,Harvey Dent is a former district attorney of G...,42


If the output is a list, `prompt` returns a `Collection`, which extends `pd.DataFrame`. To extract the `Entity` representations, use the `objects` property.

We can now store these generated objects as embeddings in a collection.

In [4]:
from promptx import store

store(*characters.objects)

This stores the object as an embedding, along with some metadata, in a vector database (ChromaDB by default). The process is quite simple, it embeds the whole object as a JSON string and each field individually. This allows us to query the database using any field in the object.

In [5]:
from promptx import query

query()

,id,type,name,description,age
0,b9feedb2-5847-4aec-8c0a-11d050bfb67a,character,Bruce Wayne,Bruce Wayne is the billionaire owner and CEO o...,38
1,5c421997-8e2b-41ac-af8a-bad8181a3555,character,Dick Grayson,Dick Grayson is Bruce Wayne's first ward and l...,25
2,9ab8ac13-408b-4b9a-b418-fe48ae1af899,character,Barbara Gordon,Barbara Gordon is the daughter of Gotham City ...,29
3,34233dc3-b115-4f76-b648-35fa41e2cc3f,character,Selina Kyle,"Selina Kyle, also known as Catwoman, is a skil...",35
4,180cfd37-efcb-4431-9334-c87e246030f6,character,Harvey Dent,Harvey Dent is a former district attorney of G...,42


Now let's generate some more characters and add them to the collection. We'll first get any existing characters and extract their names, which we can pass to the prompt to avoid generating duplicates. Any characters generated will be added the list during iteration. Finally, we'll store all the generated characters in the collection.

In [6]:
n = 3
characters = query().objects

for _ in range(n):
    characters += prompt(
        '''
        Generate a list of new characters from the Batman universe.
        Don't use any of the existing characters.
        ''',
        input = {
            'existing_characters': [c.name for c in characters],
        },
        output=[Character],
    ).objects

store(*characters)
query()

/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; e

,id,type,name,description,age
0,b9feedb2-5847-4aec-8c0a-11d050bfb67a,character,Bruce Wayne,Bruce Wayne is the billionaire owner and CEO o...,38
1,5c421997-8e2b-41ac-af8a-bad8181a3555,character,Dick Grayson,Dick Grayson is Bruce Wayne's first ward and l...,25
2,9ab8ac13-408b-4b9a-b418-fe48ae1af899,character,Barbara Gordon,Barbara Gordon is the daughter of Gotham City ...,29
3,34233dc3-b115-4f76-b648-35fa41e2cc3f,character,Selina Kyle,"Selina Kyle, also known as Catwoman, is a skil...",35
4,180cfd37-efcb-4431-9334-c87e246030f6,character,Harvey Dent,Harvey Dent is a former district attorney of G...,42
5,2942e41d-6df0-4824-859c-0a6c6134ad16,character,Cassandra Kane,Cassandra Kane is a highly skilled martial art...,20
6,00a2876a-7fd7-4db1-8396-d1b131f0a22d,character,Lucas Fox,Lucas Fox is a technological genius and the so...,25
7,9bd5d62d-add1-4b24-a344-acd69ed36b3b,character,Stephanie Brown,Stephanie Brown is a skilled acrobat and fight...,22
8,108f5dc3-9575-4eda-9fee-e30bcb67fae2,character,Kate Kane,Kate Kane is a wealthy heiress and skilled fig...,28
9,fc672ca6-78b3-424b-98b3-5d9e2201384a,character,Duke Thomas,Duke Thomas is a young man who eventually beco...,19


Now that the characters are embedded, we can query the collection.

In [7]:
villains = query('they are a villain')
villains

,id,type,name,description,age
0,180cfd37-efcb-4431-9334-c87e246030f6,character,Harvey Dent,Harvey Dent is a former district attorney of G...,42
1,17507a2f-2377-4517-9130-d0769c856e2d,character,Grace Ramirez,Grace Ramirez is a former police officer who w...,32
2,74f86d95-5198-4163-bbce-56ee41de5990,character,Oliver Marks,Oliver Marks is a skilled martial artist who j...,30
3,2726f538-da77-4a74-9d91-c16b72e58635,character,Alexis Stone,Alexis Stone is a tough and intelligent detect...,35
4,5032325e-719a-4bfa-8960-4415baa2c3cf,character,Luna Shadow,Luna Shadow is a mysterious and agile characte...,25
5,34233dc3-b115-4f76-b648-35fa41e2cc3f,character,Selina Kyle,"Selina Kyle, also known as Catwoman, is a skil...",35
6,ab351000-b62f-41a6-8d23-8380296a0f00,character,Evelyn Pierce,Evelyn Pierce is a talented hacker and member ...,28


This compares the query text with the stored objects, returning results that are closest in vector space.

*Note: the effectiveness of embedding queries will depend on what data has been embedded. In this case, ChatGPT will know some details about the generated characters and so does a decent job on this data. For other data, you may find generating synthetic intermediary data to be helpful. E.g. generating `thoughts` and/or `quotes` about a set of documents.*

Because `Collection` extends `pd.DataFrame`, we can use all the usual Pandas methods to filter and sort the results.

In [8]:
villains[villains.age < 30]

,id,type,name,description,age
4,5032325e-719a-4bfa-8960-4415baa2c3cf,character,Luna Shadow,Luna Shadow is a mysterious and agile characte...,25
6,ab351000-b62f-41a6-8d23-8380296a0f00,character,Evelyn Pierce,Evelyn Pierce is a talented hacker and member ...,28


Relationships can be defined by setting the field to a type which subclasses `Entity` (or a list of that type). Internally, this is stored as a query and then loaded when the field is accessed from the database.

In [9]:
class StoryIdea(Entity):
    title: str
    description: str = None
    characters: list[Character] = None

characters = query('they are a villain').sample(3).objects

ideas = prompt(
    'Generate some story ideas',
    input={
        'characters': characters,
    },
    output=[StoryIdea],
).objects

for idea in ideas:
    idea.characters = characters

store(*ideas, collection='story-ideas')
query(collection='story-ideas')

/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


,id,type,title,description,characters
0,b1f20f48-d998-45ec-9154-685ec0090cc7,storyidea,The Cat and the Shadow,"Selina Kyle, aka Catwoman, teams up with Luna ...",[{'id': '34233dc3-b115-4f76-b648-35fa41e2cc3f'...
1,70f5ce7d-ec1e-4191-995f-23ec13bf0888,storyidea,The Vigilante Cop,"Grace Ramirez, a former police officer turned ...",[{'id': '34233dc3-b115-4f76-b648-35fa41e2cc3f'...
2,7f21519f-0752-4c5c-999a-6f2c1d845538,storyidea,Shadows of Gotham,"Selina Kyle, Grace Ramirez, and Luna Shadow jo...",[{'id': '34233dc3-b115-4f76-b648-35fa41e2cc3f'...


Note that the output is being stored in a collection called `story-ideas`, which is created if it doesn't exist. Previously, all the data we've stored has been in the 'default' collection.

*Collections are widely used internally to represent stored models, templates, prompt history, etc. This provides a consistent interface for accessing and manipulating data.*

So far we've used the default model (GPT-3.5) when generating data, but you can specify a custom model using the `llm=` parameter.

In [10]:
from promptx.models.openai import ChatGPT

gpt4 = ChatGPT(id='gpt4', model='gpt4')

characters = prompt(
    'Generate some characters from the Batman universe',
    output=[Character],
    llm=gpt4,
)

/Users/richardlayte/Work/layterz/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2080: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


You can define any commonly used models, templates, etc, along with defining other settings, by creating a `config.py` file in the root of the project (i.e. adjacent to the `.px/` directory). This file is loaded when the project is initialized and a `setup` function is expected. Here's a simple example that defines a few custom models and a template.

```
# ./config.py

from promptx.models.openai import ChatGPT

gpt4 = ChatGPT(id='gpt4', model='gpt4')

def setup(session):
    session.store(gpt4, collection='models')
```
